[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/show3dvolume/show3dvolume_all_features.ipynb)

# Show3DVolume -- All Features

Comprehensive demo of the orthogonal slice viewer using realistic electron tomography synthetic data.
Covers NumPy/PyTorch inputs, slice navigation, colormaps, log scale, auto contrast,
crosshairs, scale bar, non-cubic volumes, and UI toggles.

In [ ]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

In [ ]:
import numpy as np
from quantem.widget import Show3DVolume

## 1. Basic volume (NumPy) -- Core-shell nanoparticle

In [ ]:
def make_gold_nanoparticle(n=96):
    """Realistic HAADF-STEM tomographic reconstruction of a multiply-twinned gold nanoparticle."""
    z, y, x = np.mgrid[:n, :n, :n]
    cz, cy, cx = n / 2, n / 2, n / 2
    ax, ay, az = np.abs(x - cx), np.abs(y - cy), np.abs(z - cz)

    # Truncated octahedron shape (faceted, common for FCC gold nanoparticles)
    l1 = ax + ay + az
    linf = np.maximum(np.maximum(ax, ay), az)
    shape_dist = np.maximum(l1 / 1.6, linf)
    R = n * 0.37
    particle = 1.0 / (1 + np.exp((shape_dist - R) * 1.2))

    # Z-contrast density: core slightly brighter (bimetallic Au@Pd-like)
    r = np.sqrt((x - cx) ** 2 + (y - cy) ** 2 + (z - cz) ** 2)
    core_weight = 1.0 / (1 + np.exp((r - R * 0.45) * 0.6))
    density = 0.65 + 0.35 * core_weight

    # Lattice fringes (111 planes)
    d111 = 3.8
    fringes = 0.12 * (
        np.cos(2 * np.pi * (x + y + z) / (d111 * np.sqrt(3)))
        + 0.5 * np.cos(2 * np.pi * (x - y + z) / (d111 * np.sqrt(3)))
    )

    # Twin boundaries
    twin1 = 0.18 * np.exp(-((x - cx) * 0.707 + (y - cy) * 0.707) ** 2 / 2.0)
    twin2 = 0.12 * np.exp(-((x - cx) * 0.707 - (z - cz) * 0.707) ** 2 / 2.0)

    # Stacking fault
    sf = 0.10 * np.exp(-((z - cz) + 0.3 * (x - cx)) ** 2 / 1.5)

    volume = particle * (density + fringes + twin1 + twin2 + sf)

    # Internal voids
    for dx, dy, dz, vr in [(10, -6, 4, 3.0), (-8, 8, -5, 2.5), (5, -3, -12, 2.0)]:
        d = np.sqrt((x - cx - dx) ** 2 + (y - cy - dy) ** 2 + (z - cz - dz) ** 2)
        volume -= 0.5 * particle * np.exp(-(d ** 2) / (2 * vr ** 2))

    # Satellite particles
    for sx, sy, sz, sr in [
        (R * 0.85, 0, R * 0.5, 3.5),
        (-R * 0.7, R * 0.6, 0, 2.8),
        (0, -R * 0.8, -R * 0.5, 3.0),
    ]:
        d = np.sqrt((x - cx - sx) ** 2 + (y - cy - sy) ** 2 + (z - cz - sz) ** 2)
        volume += 0.8 / (1 + np.exp((d - sr) * 2.0))

    volume += np.random.normal(0, 0.015, volume.shape)
    volume = np.clip(volume, 0, None)
    return volume.astype(np.float32)


particle = make_gold_nanoparticle()
Show3DVolume(particle, title="Gold Nanoparticle", cmap="inferno")

## 2. PyTorch tensor input

In [ ]:
import torch

particle_torch = torch.from_numpy(particle)
Show3DVolume(particle_torch, title="Gold Nanoparticle (PyTorch tensor)", cmap="inferno")

## 3. Set slice positions -- navigate to internal features

Navigate to the twin boundary and an internal void.

In [ ]:
w = Show3DVolume(particle, title="Slice Through Twin Boundary", cmap="inferno")
# Position to see twin boundary and void
w.slice_z = 48  # center XY plane
w.slice_y = 42  # through void region
w.slice_x = 58  # through twin boundary
w

## 4. Colormaps

In [ ]:
Show3DVolume(particle, title="Viridis colormap", cmap="viridis")

In [ ]:
Show3DVolume(particle, title="Grayscale colormap", cmap="gray")

## 5. Log scale -- Porous catalyst support

Mesoporous support with bright metal nanoparticle deposits creates high dynamic range
that benefits from log-scale visualization.

In [ ]:
def make_porous_catalyst(nz=48, ny=48, nx=48):
    """Porous catalyst support -- like mesoporous silica."""
    z, y, x = np.mgrid[:nz, :ny, :nx]
    volume = np.zeros((nz, ny, nx))
    # Random pore network
    np.random.seed(42)
    n_pores = 30
    for _ in range(n_pores):
        px, py, pz = np.random.rand(3) * [nx, ny, nz]
        pr = 2 + np.random.rand() * 4
        d = np.sqrt((x - px) ** 2 + (y - py) ** 2 + (z - pz) ** 2)
        volume += np.exp(-d ** 2 / (2 * pr ** 2))
    # Invert: pores are low, walls are high
    volume = 1.0 - volume / volume.max()
    volume = np.clip(volume, 0, 1)
    # Metal nanoparticles deposited in pores (bright spots)
    for _ in range(8):
        px, py, pz = np.random.rand(3) * [nx, ny, nz]
        d = np.sqrt((x - px) ** 2 + (y - py) ** 2 + (z - pz) ** 2)
        volume += 3.0 * np.exp(-d ** 2 / (2 * 1.5 ** 2))
    return volume.astype(np.float32)


catalyst = make_porous_catalyst()
Show3DVolume(catalyst, title="Porous Catalyst (log scale)", cmap="inferno", log_scale=True)

Compare with linear scale to see the difference:

In [ ]:
Show3DVolume(catalyst, title="Porous Catalyst (linear scale)", cmap="inferno", log_scale=False)

## 6. Auto contrast -- handle outlier intensities

A volume with a single hot pixel demonstrates percentile-based contrast stretching.

In [ ]:
particle_outlier = particle.copy()
particle_outlier[48, 48, 48] = 50.0

Show3DVolume(
    particle_outlier,
    title="With outlier -- auto contrast ON",
    cmap="inferno",
    auto_contrast=True,
)

In [ ]:
Show3DVolume(
    particle_outlier,
    title="With outlier -- auto contrast OFF (washed out)",
    cmap="inferno",
    auto_contrast=False,
)

## 7. Crosshair toggle

In [ ]:
Show3DVolume(
    particle,
    title="Crosshairs visible",
    cmap="inferno",
    show_crosshair=True,
)

In [ ]:
Show3DVolume(
    particle,
    title="Crosshairs hidden",
    cmap="inferno",
    show_crosshair=False,
)

## 8. Scale bar -- realistic pixel size

Typical HAADF-STEM tomography at 2.0 angstrom/pixel.

In [ ]:
Show3DVolume(
    particle,
    title="Core-Shell Particle (2.0 A/px)",
    cmap="inferno",
    pixel_size_angstrom=2.0,
    scale_bar_visible=True,
)

## 9. Non-cubic volume -- Grain boundary analysis

Two crystalline grains meeting at an amorphous boundary.
The volume is intentionally non-cubic (32 x 64 x 96) to test proper handling of anisotropic shapes.

In [ ]:
def make_grain_boundary(nz=32, ny=64, nx=96):
    """Two crystalline grains meeting at a boundary."""
    z, y, x = np.mgrid[:nz, :ny, :nx]
    # Grain 1 (left)
    freq1 = 0.15
    grain1 = (
        np.cos(2 * np.pi * freq1 * x)
        * np.cos(2 * np.pi * freq1 * y)
        * np.cos(2 * np.pi * freq1 * z)
    )
    # Grain 2 (right, different orientation)
    angle = np.pi / 5
    grain2 = np.cos(
        2 * np.pi * freq1 * (x * np.cos(angle) + y * np.sin(angle))
    ) * np.cos(2 * np.pi * freq1 * z)
    # Boundary in the middle
    boundary_x = nx // 2
    blend = 1.0 / (1 + np.exp(-(x - boundary_x) * 0.5))
    volume = grain1 * (1 - blend) + grain2 * blend
    # Amorphous boundary region
    boundary_region = np.exp(-((x - boundary_x) ** 2) / (2 * 3 ** 2))
    volume += boundary_region * np.random.normal(0, 0.5, volume.shape)
    volume += np.random.normal(0, 0.05, volume.shape)
    return volume.astype(np.float32)


grains = make_grain_boundary()
print(f"Volume shape (non-cubic): {grains.shape}")
Show3DVolume(grains, title="Grain Boundary (32 x 64 x 96)", cmap="viridis")

## 10. Hide controls and stats -- minimal view

In [ ]:
Show3DVolume(
    particle,
    title="Minimal view",
    cmap="inferno",
    show_controls=False,
    show_stats=False,
    show_crosshair=False,
)

## 11. Custom dimension labels

Configure axis names to match your data conventions.

In [ ]:
Show3DVolume(
    particle,
    title="Custom Axis Labels",
    cmap="inferno",
    dim_labels=["depth", "row", "col"],
)